<h1 style='text-align: center'>Exemplo de aprendizado de uma rede neural de múltiplas camadas</h1>

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from sklearn.model_selection import KFold

import random

<h2>Load Data</h2>

In [2]:
df = pd.read_csv('diabetes.csv', header=None)
df.describe()

,0,1,2,3,4,5,6,7,8
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [3]:
df_array = np.array(df)

dados = df_array[:, :-1]
classes = df_array[:, -1]

In [4]:
dados.shape, classes.shape

((768, 8), (768,))

In [5]:
min_values = np.min(dados, axis=0)
max_values = np.max(dados, axis=0)

In [6]:
min_values, max_values

(array([ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.078, 21.   ]),
 array([ 17.  , 199.  , 122.  ,  99.  , 846.  ,  67.1 ,   2.42,  81.  ]))

In [7]:
scaled_dados = (dados - min_values) / max_values

In [8]:
np.count_nonzero(classes == 0), np.count_nonzero(classes == 1)

(500, 268)

In [9]:
class1_data = random.choices(scaled_dados[classes == 1], k=(500-268))

In [10]:
scaled_dados = np.concatenate([scaled_dados, np.array(class1_data)], axis=0)
classes = np.hstack([classes, np.ones((500-268,))])

In [11]:
scaled_dados.shape, classes.shape

((1000, 8), (1000,))

In [12]:
idx = np.random.permutation(len(scaled_dados))
x, y = scaled_dados[idx], classes[idx]

<h1>Create Network</h1>

In [13]:
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[np.random.random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[np.random.random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [14]:
network = initialize_network(8, 100, 1)

<h1>Train Network</h1>

In [15]:
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

In [16]:
def transfer(activation):
    return 1.0 / (1.0 + np.exp(-activation))

In [17]:
def transfer_derivative(output):
    return output * (1.0 - output)

In [18]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
    
    train_data, train_labels = train
    for epoch in tqdm_notebook(range(n_epoch)):
        for i in tqdm_notebook(range(len(train_data))):
            row = train_data[i]
            label = train_labels[i]
            outputs = forward_propagate(network, row)
            expected = [label]
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)

In [19]:
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [20]:
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [21]:
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

In [22]:
def predict(network, row):
    outputs = forward_propagate(network, row)
    return np.round(outputs)

In [23]:
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
    _, n_inputs = train[0].shape
    n_outputs = 1

    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    predictions = list()
    for row in test[0]:
        prediction = predict(network, row)
        predictions.append(prediction)
    return np.hstack(predictions)

In [24]:
def cross_validation_split(X, y, n_folds):
    
    kf = KFold(n_splits=n_folds, random_state=42, shuffle=False)
    folds = []
    
    for train_index, test_index in kf.split(dados):
        aux = {}
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        aux['train'] = [X_train, y_train]
        aux['test'] = [X_test, y_test]
        
        folds.append(aux)
    
    return folds

In [25]:
def evaluate_algorithm(dados, classes, algorithm, n_folds, *args):
    folds = cross_validation_split(dados, classes, n_folds)
    scores = list()
    networks = list()
    pbar = tqdm_notebook(total=n_folds)
    for fold in folds:
        train_set = fold['train']
        test_set = fold['test']
        
        predicted = algorithm(train_set, test_set, *args)
        actual = test_set[1]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
        pbar.update(1)
        
    pbar.reset()
    return scores

In [26]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [27]:
n_folds = 10
l_rate = 0.3
n_epoch = 10
n_hidden = 5
scores = evaluate_algorithm(scaled_dados, classes, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

/home/padilha/py36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/home/padilha/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/home/padilha/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.




Scores: [70.12987012987013, 79.22077922077922, 70.12987012987013, 68.83116883116884, 75.32467532467533, 75.32467532467533, 81.81818181818183, 81.81818181818183, 76.31578947368422, 78.94736842105263]
Mean Accuracy: 75.786%
